# Intensity Preidiction 

This notebook is prepared to be run in Google [Colaboratory](https://colab.research.google.com/). In order to train the model faster, please change the runtime of Colab to use Hardware Accelerator, either GPU or TPU.

This notebook presents a short walkthrough the process of reading a dataset and training a model for retention time prediction. The dataset is an example dataset extracted from a ProteomTools dataset generated in the **Chair of Bioanalytics** at the **Technical University of Munich**.

The framework being used is a custom wrapper on top of Keras/TensorFlow. The working name of the package is for now DLOmix -  `dlomix`.

In [147]:
import numpy
import functools
import dlomix.losses as losses
import tensorflow as tf


def reshape_dims(array):
    n, dims = array.shape
    assert dims == 174
    nlosses = 1
    return array.reshape(
        [array.shape[0], 30 - 1, 2, nlosses, 3]
    )


def reshape_flat(array):
    s = array.shape
    flat_dim = [s[0], functools.reduce(lambda x, y: x * y, s[1:], 1)]
    return array.reshape(flat_dim)


def normalize_base_peak(array):
    # flat
    maxima = array.max(axis=1)
    array = array / maxima[:, numpy.newaxis]
    return array


def mask_outofrange(array, lengths, mask=-1.):
    # dim
    for i in range(array.shape[0]):
        array[i, lengths[i] - 1 :, :, :, :] = mask
    return array


def mask_outofcharge(array, charges, mask=-1.):
    # dim
    for i in range(array.shape[0]):
        if charges[i] < 3:
            array[i, :, :, :, charges[i] :] = mask
    return array


def get_spectral_angle(true, pred, batch_size=600):

    n = true.shape[0]
    sa = numpy.zeros([n])

    def iterate():
        if n > batch_size:
            for i in range(n // batch_size):
                true_sample = true[i * batch_size : (i + 1) * batch_size]
                pred_sample = pred[i * batch_size : (i + 1) * batch_size]
                yield i, true_sample, pred_sample
            i = n // batch_size
            yield i, true[(i) * batch_size :], pred[(i) * batch_size :]
        else:
            yield 0, true, pred

    for i, t_b, p_b in iterate():
        tf.compat.v1.reset_default_graph()
        with tf.compat.v1.Session() as s:
            sa_graph = losses.masked_spectral_distance(t_b, p_b)
            sa_b = 1 - s.run(sa_graph)
            sa[i * batch_size : i * batch_size + sa_b.shape[0]] = sa_b
    sa = numpy.nan_to_num(sa)
    return sa


def normalize_predictions(data, batch_size=600):
    assert "sequences" in data
    assert "intensities_pred" in data
    assert "precursor_charge_onehot" in data

    sequence_lengths = data["sequences"].apply(lambda x: len(x))
    intensities =  np.stack(data["intensities_pred"].to_numpy()).astype(np.float32)
    precursor_charge_onehot = np.stack(data["precursor_charge_onehot"].to_numpy())
    charges = list(precursor_charge_onehot.argmax(axis=1) + 1)

    intensities[intensities < 0] = 0
    intensities = reshape_dims(intensities)
    intensities = mask_outofrange(intensities, sequence_lengths)
    intensities = mask_outofcharge(intensities, charges)
    intensities = reshape_flat(intensities)
    m_idx = intensities == -1
    intensities = normalize_base_peak(intensities)
    intensities[m_idx] = -1
    data["intensities_pred"] = intensities

    if "intensities_raw" in data:
        data["spectral_angle"] = get_spectral_angle(
            np.stack(data["intensities_raw"].to_numpy()).astype(np.float32), intensities, batch_size=batch_size
        )
    return data


In [1]:
# install the DLOmix package in the current environment using pip

!python -m pip install -q dlomix==0.0.3

The available modules in the framework are as follows:

In [1]:
import numpy as np
import pandas as pd
import dlomix
from dlomix import constants, data, eval, layers, models, pipelines, reports, utils
print([x for x in dir(dlomix) if not x.startswith("_")])

['META_DATA', 'constants', 'data', 'eval', 'layers', 'models', 'pipelines', 'reports', 'utils']


- `constants`: constants to be used in the framework (e.g. Aminoacid alphabet mapping)
- `data`:  classes for representing dataset, wrappers around TensorFlow Dataset
- `eval`: custom evaluation metrics implemented in Keras/TF to work as `metrics` for model training
- `layers`: custom layer implementation required for the different models
- `models`: different model implementations for Retention Time Prediction
- `pipelines`: complete pipelines to run a task (e.g. Retention Time prediction)
- `utils`: helper modules

**Note**: reports and pipelines are work-in-progress, some funtionalities are not complete.

## 1. Load Data

We can import the dataset class and create an object of type `RetentionTimeDataset`. This object wraps around TensorFlow dataset objects for training+validation or for testing. This can be specified by the arguments `val_ratio` and `test`.

In [2]:
from dlomix.data import IntensityDataset

In [22]:
TRAIN_DATAPATH = 'D:/Compmass/proteomeTools_train_val.csv'
BATCH_SIZE = 64

int_data = IntensityDataset(data_source=TRAIN_DATAPATH,
                              seq_length=30, batch_size=BATCH_SIZE,collision_energy_col='collision_energy', val_ratio=0.2, test=False)

<class 'pandas.core.series.Series'>
0        [0, 0, 1, 0, 0, 0]
1        [0, 1, 0, 0, 0, 0]
2        [0, 0, 1, 0, 0, 0]
3        [0, 0, 1, 0, 0, 0]
4        [0, 0, 1, 0, 0, 0]
                ...        
18259    [0, 1, 0, 0, 0, 0]
18260    [0, 1, 0, 0, 0, 0]
18261    [0, 0, 1, 0, 0, 0]
18262    [0, 1, 0, 0, 0, 0]
18263    [0, 0, 1, 0, 0, 0]
Name: precursor_charge_onehot, Length: 18264, dtype: object


Now we have an RT dataset that can be used directly with standard or custom `Keras` models. This wrapper contains the splits we chose when creating it. In our case, they are training and validation splits. To get the TF Dataset, we call the attributes `.train_data` and `.val_data`. The length is now in batches (i.e. `total examples = batch_size x len`)

In [11]:
 "Training examples", BATCH_SIZE * len(int_data.train_data)

('Training examples', 14656)

In [12]:
"Validation examples", BATCH_SIZE * len(int_data.val_data)

('Validation examples', 3712)

## 2. Model

We can now create the model. We will use a simple Prediction with a conv1D encoder. It has the default working arguments, but most of the parameters can be customized.

**Note**: Important is to ensure that the padding length used for the dataset object is equal to the sequence length passed to the model.

In [13]:
from dlomix.models import PrositIntensityPredictor

In [14]:
model = PrositIntensityPredictor(seq_length=30)

## 3. Training

We can then train the model like a standard Keras model. The training parameters here are from Prosit, but other optimizer parameters can be used.  

In [19]:
#imports

import tensorflow as tf
from dlomix.eval import TimeDeltaMetric
from dlomix.losses import masked_spectral_distance

In [20]:
# create the optimizer object
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

# compile the model  with the optimizer and the metrics we want to use, we can add our custom timedelta metric
model.compile(optimizer=optimizer,
              loss=masked_spectral_distance,
              metrics=['mse'])

We store the result of training so that we can explore the metrics and the losses later. We specify the number of epochs for training and pass the training and validation data as previously described.

In [23]:
history = model.fit(int_data.train_data,
                    validation_data=int_data.val_data,
                    epochs=1)

Epoch 1/15
229/229 [==============================] - ETA: 0s - loss: 0.6117 - mse: 1.9724WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'list'> input: [<tf.Tensor 'prosit_intensity_predictor/Cast:0' shape=(None, 1) dtype=float32>, <tf.Tensor 'prosit_intensity_predictor/Cast_1:0' shape=(None, 6) dtype=float32>]
Consider rewriting this model with the Functional API.
encoded sequence:  Tensor("prosit_intensity_predictor/string_lookup/Identity:0", shape=(None, 30), dtype=int64)
Consider rewriting this model with the Functional API.
229/229 [==============================] - 119s 518ms/step - loss: 0.6117 - mse: 1.9724 - val_loss: 0.5544 - val_mse: 2.5445
Epoch 2/15
 11/229 [>.............................] - ETA: 1:45 - loss: 0.5634 - mse: 1.4458

KeyboardInterrupt: 

## 3. Testing and Reporting

We can create a test dataset to test our model. Additionally, we can use the reporting module to produce plots and evaluate the model.

Note: the reporting module is still in progress and some functionalities might easily break.

In [45]:
# create the dataset object for test data

TEST_DATAPATH = 'D:/Compmass/proteomeTools_test.csv'

test_int_data = IntensityDataset(data_source=TEST_DATAPATH,
                              seq_length=30, collision_energy_col='collision_energy',batch_size=32, test=True)

<class 'pandas.core.series.Series'>
0        [0, 0, 1, 0, 0, 0]
1        [0, 1, 0, 0, 0, 0]
2        [0, 0, 1, 0, 0, 0]
3        [0, 0, 1, 0, 0, 0]
4        [0, 0, 1, 0, 0, 0]
                ...        
18259    [0, 1, 0, 0, 0, 0]
18260    [0, 1, 0, 0, 0, 0]
18261    [0, 0, 1, 0, 0, 0]
18262    [0, 1, 0, 0, 0, 0]
18263    [0, 0, 1, 0, 0, 0]
Name: precursor_charge_onehot, Length: 18264, dtype: object


In [46]:
# use model.predict from keras directly on the testdata

predictions = model.predict(test_int_data.test_data)

In [56]:
predictions_df = pd.DataFrame()

In [104]:
predictions

array([[ 0.00151483, -0.00016489, -0.00027107, ..., -0.02478559,
        -0.02298378, -0.01914065],
       [ 0.00018149, -0.00043896, -0.0032556 , ..., -0.03871916,
        -0.03681742, -0.03799452],
       [ 0.00161823, -0.00018393, -0.00028721, ..., -0.02620803,
        -0.02486783, -0.02045185],
       ...,
       [ 0.00161323, -0.00018112, -0.00029086, ..., -0.02497605,
        -0.0238777 , -0.01930104],
       [ 0.0002696 , -0.00042114, -0.00283213, ..., -0.03069953,
        -0.03000446, -0.02962921],
       [ 0.00234061, -0.0003035 , -0.0004592 , ..., -0.03357218,
        -0.03152551, -0.02862653]], dtype=float32)

In [149]:
predictions_df['sequences'] = test_rtdata.sequences
predictions_df['intensities_pred'] = predictions.tolist()
predictions_df['precursor_charge_onehot'] = test_rtdata.precursor_charge.tolist()
predictions_df['intensities_raw'] = test_rtdata.intensities.tolist()


In [150]:
predictions_acc = normalize_predictions(predictions_df)

In [151]:
predictions_acc['spectral_angle'].describe()

count    18264.000000
mean         0.487196
std          0.214041
min          0.049291
25%          0.327691
50%          0.449062
75%          0.598523
max          0.956674
Name: spectral_angle, dtype: float64